In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 필수 라이브러리 가져오기

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor

# StratifiedKFold : 분류 작업 할 때
# KFold : 수치 작업 할 때
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, StratifiedKFold, KFold

import numpy as np
import pandas as pd 
from scipy.stats import uniform, randint

# 데이터 가져오기

In [6]:
#train = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
#test = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")

train =  pd.read_csv('datasets/train.csv')
test = pd.read_csv('datasets/test.csv')

train.shape, test.shape

((1460, 81), (1459, 80))

## 데이터 확인

In [7]:
train.head(1)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500


In [8]:
test.head(1)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal


# 피처 엔지니어링

## ID 제거

In [9]:
train_ID = train.pop('Id')
test_ID = test.pop('Id')

train.shape, test.shape

((1460, 80), (1459, 79))

## 독립변수와 종속변수 분리

In [10]:
# y = train['SalePrice']
# log 변환
y = np.log1p(train['SalePrice'])
X = train.drop('SalePrice', axis=1)

## 컬럼 분리

In [11]:
cat_cols = X.select_dtypes(exclude = np.number).columns.tolist()
num_cols = X.select_dtypes(include = np.number).columns.tolist()

print(cat_cols, num_cols)

['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition'] ['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'Enclos

# 모델 만들기

## 훈련 및 검증 데이터셋 분리

In [12]:
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42
)

X_tr.shape, X_val.shape, y_tr.shape, y_val.shape

((1022, 79), (438, 79), (1022,), (438,))

## 파이프라인 모델 만들기

## 머신러닝 모델 시간 측정 함수

In [13]:
import time

def execution_time(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        elapsed_time = end_time - start_time
        hours, rem = divmod(elapsed_time, 3600)
        minutes, seconds = divmod(rem, 60)
        print(f"Execution time for {func.__name__}: {int(hours)}h {int(minutes)}m {seconds:.2f}s")
        return result
    return wrapper

## Log Transformation

In [14]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor

# StratifiedKFold : 분류 작업 할 때
# KFold : 수치 작업 할 때
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, StratifiedKFold, KFold

import numpy as np
from scipy.stats import uniform, randint

import time

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, num_cols),
        ('cat', categorical_transformer, cat_cols)
    ])

"""
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestRegressor(random_state=42))
])
"""

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),  # Assuming preprocessor is already defined
    ('classifier', LGBMRegressor(random_state=42, verbose=-1))
])

# Define the parameter distributions
param_distributions = {
    'classifier__learning_rate': uniform(0.01, 0.2),
    'classifier__max_depth': randint(3, 15),
    'classifier__reg_alpha': uniform(0.0, 1.0),
    'classifier__reg_lambda': uniform(0.0, 1.0),
}

split_number = 5
# stratified_kfold, 분류모형 만들 때
# stratified_kfold = StratifiedKFold(n_splits=split_number, shuffle=True, random_state=42)

kfold = KFold(n_splits=split_number, shuffle=True, random_state=42)

random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_distributions,
    n_iter=50,
    cv=kfold, # stratified_kfold, 분류모형 만들 때
    scoring='neg_mean_squared_error',
    random_state=42,
    n_jobs=-1
)

@execution_time
def model_builing(model, X_train, y_train):
    model.fit(X_train, y_train)
    best_model = model.best_estimator_
    return best_model

In [15]:
best_model = model_builing(random_search, X_tr, y_tr)

Execution time for model_builing: 0h 0m 26.03s


## 모형 평가

In [16]:
from sklearn.metrics import mean_squared_error
import numpy as np

def get_score(model, X_tr, X_val, y_tr, y_val):
    tr_pred = model.predict(X_tr)
    val_pred = model.predict(X_val)
    tr_score = np.sqrt(mean_squared_error(y_tr, tr_pred))
    val_score = np.sqrt(mean_squared_error(y_val, val_pred))
    return f"train: {tr_score}, validation: {val_score}"

get_score(best_model, X_tr, X_val, y_tr, y_val)

'train: 0.07056553538855749, validation: 0.12953821614970773'

# 평가 제출

In [17]:
train.head(1)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500


In [20]:
#submission = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv")
submission = pd.read_csv("datasets/sample_submission.csv")
# 로그 역변환
final_preds = np.expm1(best_model.predict(test))
submission['SalePrice'] = final_preds
submission.head(1)

,Id,SalePrice
0,1461,127549.393936


In [16]:
#submission.to_csv("submission.csv", index=False)